In [5]:
from google.colab import files
uploaded = files.upload()

Saving model_epoch_31.pth to model_epoch_31.pth


In [6]:
import os
import torch
import torch.nn.functional as F
from diff_model import nn_model
from helpers import show_image
from torch.utils.checkpoint import checkpoint as chkpt

input dim 1 128
input dim 512 128
input dim 1 128


In [7]:
device = (
    torch.device("cuda")
    if torch.cuda.is_available()
    else torch.device("mps")
    if torch.backends.mps.is_available()
    else torch.device("cpu")
)
# save_dir = "weights/data_context/"
current_directory = os.getcwd()
print(current_directory)
save_dir = current_directory

/content


In [8]:
# load in model weights and set to eval mode
model_path = os.path.join(save_dir + "/model_epoch_31.pth")
print("curr model path", model_path)

curr model path /content/model_epoch_31.pth


In [9]:
print("model path", model_path)
checkpoint = torch.load(f=model_path, map_location=device)

model path /content/model_epoch_31.pth


<ipython-input-9-a0b6c5c254bc>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(f=model_path, map_location=device)


In [10]:
nn_model.load_state_dict(checkpoint["model_state_dict"])
nn_model.eval()
print("Loaded in Model")

Loaded in Model


In [11]:
def ddim_sample(nn_model, n_samples, timesteps, alphas_cumprod, eta=0.5, device="cuda"):
    """
    DDIM Sampling for a diffusion model.

    Args:
        nn_model: Trained neural network model.
        n_samples: Number of samples to generate.
        timesteps: Total diffusion steps.
        alphas_cumprod: Cumulative product of alphas (beta schedule).
        eta: Controls the amount of noise injected during sampling. Default is 0 for deterministic sampling.
        device: Device to run sampling on, 'cuda' or 'cpu'.

    Returns:
        A batch of generated samples.
    """
    # Start with pure noise
    x = torch.randn(
        (n_samples, 3, 128, 128), device=device
    )  # Adjust image shape as per your dataset (e.g., 3x128x128)
    seg_mask = torch.randn((n_samples, 1, 128,128), device=device)
    text_emb = torch.randn((n_samples, 512), device=device)

    # Reverse sampling steps
    with torch.autocast(device_type='cuda'):
        for t in reversed(range(1, timesteps)):
            t_tensor = torch.full((n_samples,), t, device=device, dtype=torch.long)

            # Predict noise using the neural network
            pred_noise = nn_model(x, t_tensor / timesteps, text_emb, seg_mask)
            print('pred noise', pred_noise)
            # Calculate x_t_minus_1 (previous step sample)
            eps = 1e-8
            alpha_t = alphas_cumprod[t]
            alpha_t = torch.clamp(alpha_t, eps)
            alpha_t_prev = alphas_cumprod[t - 1]
            beta_t = 1 - alpha_t
            print('vals', alpha_t, beta_t)
            sigma_t = eta * torch.sqrt(beta_t)

            print('val of sigma_t', sigma_t)
            # Compute the next step sample
            pred_x0 = (x - torch.sqrt(1 - (alpha_t + eps)) * pred_noise) / torch.sqrt(alpha_t+eps)
            noise = sigma_t * torch.randn_like(x) if t > 1 else torch.zeros_like(x)

            print('pred x0', pred_x0)
            print('noise', noise)
            # Sample the next step
            x = torch.sqrt(alpha_t_prev) * pred_x0 + torch.sqrt(1 - alpha_t_prev) * noise

            show_image(x[0], title=f"After denoising step {t}")

            del t_tensor, pred_noise
            torch.cuda.empty_cache()
    return x

In [12]:
# Example usage:
n_samples = 1
timesteps = 500  # Total timesteps used in the model
beta_start = 1e-4
beta_end = 0.02
alphas_cumprod = torch.cumprod(
    torch.linspace(beta_start, beta_end, timesteps), dim=0
)  # Adjust based on your noise schedule

In [13]:
samples = ddim_sample(nn_model, n_samples, timesteps, alphas_cumprod, eta=0.5)

down 1 shape torch.Size([1, 64, 64, 64])
down2 shape torch.Size([1, 128, 32, 32])
seg input torch.Size([1, 1, 128, 128]) 64
seg embeds torch.Size([1, 128, 128, 128])
down 2 input dtype and device cuda:0 torch.float16
attn output 111----- torch.Size([1, 4096, 512])
attn output----- layer norm torch.Size([1, 128, 64, 64]) tensor(0.1317, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>) tensor(0.9297, device='cuda:0', dtype=torch.float16, grad_fn=<StdBackward0>)
attn output torch.Size([1, 128, 64, 64])
layer up down2 device -- cuda:0
up1 shape torch.Size([1, 128, 64, 64]), down1 torch.Size([1, 64, 64, 64]), up_down2 torch.Size([1, 128, 64, 64]), down2 torch.Size([1, 128, 32, 32]), combined embeds torch.Size([1, 128, 64, 64])
up2 shape torch.Size([1, 64, 128, 128])
attn out adjusted torch.Size([1, 64, 64, 64])
combined embeds adjust torch.Size([1, 64, 64, 64])
downsampled up2 torch.Size([1, 64, 64, 64])
combined torch.Size([1, 64, 64, 64])
up3 tensor shape torch.Size([1, 64, 1

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


TypeError: 'Tensor' object is not callable

In [ ]:
def pndm_sample(nn_model, n_samples, timesteps, alphas_cumprod, device="cuda"):
    """
    PNDM Sampling for a diffusion model.

    Args:
        nn_model: Trained neural network model.
        n_samples: Number of samples to generate.
        timesteps: Total diffusion steps.
        alphas_cumprod: Cumulative product of alphas (beta schedule).
        device: Device to run sampling on, 'cuda' or 'cpu'.

    Returns:
        A batch of generated samples.
    """
    # Start with pure noise
    x = torch.randn(
        (n_samples, 3, 128, 128), device=device
    )  # Adjust image shape as per your dataset

    # Define step size (can be adjusted for quality vs speed)
    step_size = timesteps // 4  # Skip 4 steps at a time

    for i, t in enumerate(reversed(range(1, timesteps, step_size))):
        t_tensor = torch.full((n_samples,), t, device=device, dtype=torch.long)

        # Predict noise using the neural network
        pred_noise = nn_model(x, t_tensor / timesteps)

        # Compute the cumulative product of alpha_t and the previous one
        alpha_t = alphas_cumprod[t]
        alpha_t_prev = (
            alphas_cumprod[t - step_size] if t - step_size >= 0 else alphas_cumprod[0]
        )

        # Calculate sigma for noise injection
        sigma_t = torch.sqrt(1 - alpha_t)

        # Reconstruct the next x using noise prediction
        pred_x0 = (x - sigma_t * pred_noise) / torch.sqrt(alpha_t)

        # Use the difference between alpha_t and alpha_t_prev to correct x
        x = (
            torch.sqrt(alpha_t_prev) * pred_x0
            + torch.sqrt(1 - alpha_t_prev) * pred_noise
        )

        show_image((samples[0]), title=f"After denoising step {i}")

    return x

In [ ]:
# Example usage:
samples_pndm = pndm_sample(nn_model, n_samples, timesteps, alphas_cumprod)